# Functional Programming

* Math style variables. Not assigned but **bound** to a value. No side effects.
* Functions and recursion is key in computation
* All values are first order.
* Garbage collecting runtime, recursive compositions and data structures
* Lisp is first functional language. Many followed: ML, Scheme, Ocaml, Haskell, Erlang, Closure...
* Popular in academics and also in industrial applications requiring verifiable, fault tolerant systems.

Haskell is a popular functional programming language named after Haskell Curry, a mathematician worked in combinatorial logic and had many important contributions on programming languages and theory of computation.

There are two major Haskell interpreters, GHC (Glasgow Haskell Compiler) and Hugs.

## Functions

A function definition in Haskell is very simple
```haskell

funcname par1 par2 par3 =  expression

```
There is no type required, Haskell infers types itself. However you can define type of function with a ``name::type`` syntax if you like.

`:type expression` give type of the expression.

In [1]:
f x y = x*y

square::Integer->Integer
square x = x*x

:type f
:type square

f :: forall {a}. Num a => a -> a -> a

square :: Integer -> Integer

`Num a => a -> a -> a` is read as. `a` is a number class type and function gets an `a` typed value, and another `a` type value and returns an `a` type value. For the beginning we can assume it is equivalent to `Integer -> Integer -> Integer`.

Function applications do not require paranthesis. Parantheses are required for controlling precedence. Function application have higher precedence and left associative. `f square 4 5` gets `square` as first parameter, `4` as the second parameter to `f`. 

`square 4-1` is parsed as `(square 4)-1`.

In [2]:
square 1234
f (square 241512) (square 3254151)

1522756

617664770611990162081344

## Recursion

In functional programming recursion is the main tool for repeated computations. There is no loop, iteration or goto.

In order to solve problems with recursion, you need to think mathematically. Express your problem in `n` in terms of solution of the same problem in `n-1` or similar. Then write a minimum condition (or termination condition) that you can define without recursion.

For example. Find ssum (n) = sum of values in range 1 to n.

In [3]:
ssum n = if n <= 0 then
            0
         else
            n + ssum (n-1)
:t ssum
ssum 123

ssum :: forall {t}. (Ord t, Num t) => t -> t

7626

`ssum 123` is evaluated as `123 + ssum 122 = 123 + 122 + ssum 121 = 123 + 122 + ... + 1 + ssum 0 = ... + 0 = 7626`
Instead of `if then else` syntax, you can write conditions on definition as **patterns**.
```Haskell
func x 0 = exp1
func x 1 = exp2
func x n = exp3
```
will match second parameter and compute the parameter accordingly. If it matches 0 `exp1` will be the result, if matches 1, `exp2` will be returned, otherwise it will be `exp3` with `n` is substituted with the value of the parameter.

Also conditionals can be specified as guard clauses
```Haskell

func x n | n <= 0 = exp1
         | n == 1 = exp2
         | otherwise = exp3

```


In [4]:
ssum n | n <= 0 = 0
       | otherwise = n + ssum (n-1)

In [5]:
ssum' n s  | n <= 0 = s
           | otherwise =  ssum' (n-1) (n+s)
            
ssum' 123 0

7626

`ssum'` example above calculates the same sum by `ssum`. However it **accumulates** the sum in the paramater `s` as:
`ssum' 123 0 = ssum' 122 123 = ssum' 121 145 = ssum' 120 266 = ssum' 119 386 = ... = ssum' 1 7625 = ssum' 0 7626 = 7626`.

This recursion style is closer to imperative paradigm loops. The parameter keeps the current computation state, which is the current sum so far.

When a recursive call does nothing but returns the value returned by the call is called **tail recursion**. Some compilers/interpreters recognize this case and implement tail recursion more efficiently, without using a stack frame per call.

## Efficiency
Assume we need to find n$^{th}$ fibonacci value. It is simply $fib(n) = fib(n-1)+fib(n-2)$. The implementation below
is this simple definition. For example `fib 5`:
```
fib 5 = fib 4 + fib 3 = fib 3 + fib 2 + fib 3 = fib 2 + fib 1 + fib 2 + fib 3 = 
fib 1 + fib 0 + fib 1 + fib 2 + fib 3 = 1 + 1 + 1 + fib 1 + fib 0 + fib 3 = 5 + fib 2 + fib 1 =
5 + fib 1 + fib 0 + fib 1 = 8
```
Number of summations required is exponentially increasing by `n`. Our function does not remember the
`fib (n-2)` value which is also needed in `fib (n-1)` and recomputes it. 

In [6]:
fib 0 = 1
fib 1 = 1
fib n = fib (n-1) + fib (n-2)

fib 32
-- This is too slow!!!


3524578

Alternatively, we can define recursion as a growing sequence.
`1 1 2 3 5 8 11 ...`. We can seed the current two values in sequence to get the next value. 

$fib (n, v_1, v_2) = \left\{ 
           \begin{array}{ll} v_2 & \mathrm{if} \; n = 0\\
                             fib (n-1, v_2, v_1+v_2) & \mathrm{else}\\
            \end{array} \right.$ 

If $n$ recursive call will find result in the last seed value.

Let us write a helper function `fibh` that will get `n`, current repetition number `i` and two current fibonacci sequence values, `a`: fib(i-1) and `b`:fib(i) respectively. Then, in the next recursion step, `b` will be used as the first and `a+b` will be the second parameter. The sequence will grow as:
`fibh .. 1 1 = fibh .. 1 2  = fibh .. 2 3 = fibh .. 3 5 = fibh .. 5 8 = fibh .. 8 13 = ...`
We need to go `n` recursion level and return last computed sequence value as the result. We increment `i` at each recursion level and test if `i == n` for termination. The deepest level will return `b` as the value which will be propagated as the result.

In [7]:
fibh n i a b | i == n = b
             | otherwise = fibh n (i+1) b (a+b)
               
fib2 n = fibh n 1 1 1   -- initial condition: i=1, a=1, b=1 

fib2 32

3524578

## Local definitions

It is a good practice to divide long expressions to improve readability. Instead of:

`(((x-3)/4) + (y-20)/3 ) * ( (y-20)/3 * 100 / (x-3)/4 )`

One can write:
```Haskell
let newx = (x-3)/4
    newy = (y-20)/3
    scale = newy * 100 / newx
in (newx+newy)*scale
```
This way, your code will be easier to read, and also some calculations will not be repeated, `(x-3)/4` and `(y-20)/3`
are evaluated only once.

Haskell `let ... in ...` blocks allow an expression to be evaluated in existense of a sequence of local definitions. The `let` block can contain as many definition as desired including functions and variables. The value of `let` block is the expression after the `in`. The expression in the `in` part can contain references to functions and variables in `let` block. However these definitions are local. They are not valid outside of `let...in...` block. You can use it either with indentation or ; and curly braces. See https://en.wikibooks.org/wiki/Haskell/Indentation for Haskell and indentation.

In [8]:
let x = 3
    y = 4
 in x+y+23

let x = 1; y = 3; f x = x +1 in f (x+y)

let { x =1 ; y = 4;
 f x = x+1 ; z = f(x+y)} in f z

30

5

7

The definitions between `let` and `in` are local definitions that are only valid for evaluation of expression following `in`, which is the actual expression. Local definitions are not accessible outside of the `let` block.

Sometimes local definitions are crucial in efficiency. Assume you have the following power implementation:

In [9]:
pow1 x 0 = 1
pow1 x n = x * pow1 x (n-1)

pow1 0.999999 1000000

0.3678792572210609

It makes `n` levels of recursive calls. However you can make it faster if you define power in terms of square function and divide `n` by two at each step as `power x n = (power x (n/2)) * (power x (n/2))`. You need to handle the `n` being odd.

In [10]:
pow2 x 0 = 1
pow2 x n | n `mod` 2 == 0 = pow2 x (div n 2) * pow2 x (n `div` 2)
         | otherwise      = x * pow2 x (n `div` 2) * pow2 x (n `div` 2)
pow2 0.999999 1000000

Line 2: Use even
Found:
n `mod` 2 == 0
Why not:
even n

0.367879257215518

Probably it is similar or even worse because, it is repeating the same computation. 

`pow 2 8 = pow 2 4 * pow 2 4 = pow 2 2 * pow 2 2 * pow 2 4 = pow 2 1 * pow 2 1 * pow 2 1 * pow 2 1 * ....`

If we localy define repeat computations and use previously calculated values:

In [11]:
pow3 x 0 = 1
pow3 x n = let pnover2 =  pow3 x (n `div` 2)
           in if n `mod` 2 == 0 then pnover2 * pnover2
                                else pnover2 * pnover2 * x
pow3 0.999999 1000000

-- or

square x = x * x 
pow4 x 0 = 1
pow4 x n | n `mod` 2 == 0 = square (pow4 x (n `div` 2))
         | otherwise      = x * square (pow4 x (n `div` 2))
pow4 0.999999 1000000

Line 3: Use even
Found:
n `mod` 2 == 0
Why not:
even nLine 11: Use even
Found:
n `mod` 2 == 0
Why not:
even n

0.367879257215518

0.367879257215518

This new definition will call $\lceil log_2 n \rceil$ recursion levels. As `n` gets larger, the difference will be more significant. For example: 

$\lceil log_2 10^{10} \rceil = 40$ vs $10^{10}$.


Local definitions can be arbitrarily nested. Each definition only spans its lexical scope:

In [12]:
let f x = x*x
    y = 4
 in (let f x = x+1
         z = 3
         x = 10
      in (let y = 2
           in f x * y + z) + f y + z) + f y

49

Let us put indexes under variables to indicate the correspondence:
```Haskell
let f1 x = x*x
    y1 = 4
 in (let f2 x = x+1
         z1 = 3
         x1 = 10
      in (let y2 = 2
           in f2 x * y2 + z1) + f2 y1 + z1) + f1 y1

```
        

## User Defined Types

In Haskell new data types can be defined by `type` and `data` keywords. `type` is used to define aliases to existing types where `data` creates new data types with **type constructor tags** and disjoint union operation.

In [13]:
type Person = (String, Integer, Bool)

data WState = Sunny | PartClouds| Clouds | LightRain| Rain | LightSnow | Snow | IceRain    deriving (Show,Eq)  -- this part is just for Haskell to show the values, no other meaning

type Forecast = (WState, Float, Float)


today = (Rain, -1.5 , 10)

nextday = (PartClouds, 0, 12)::Forecast   -- ::Forecast specifies the data type

:t today

:t nextday

today == nextday

today :: forall {b} {c}. (Fractional b, Num c) => (WState, b, c)

nextday :: Forecast

False

Even though `today` and `nextday` looks like they have different types, they have the same type. `WState` is a data type which has one of the values in `|` separated constructors list. Each name like `Sunny, Rain,...` is a value constructor or tag that creates a value of type `WState`.

Constructors can also get other values as arguments, as if they are functions.

`data Value = Rat (Integer,Integer) | RV Float `

In this type definition a value can be either represented as division of two integer, or it is an integer or it is a floating point value. The value representation is tagged with its constructor. Pattern matching in function definitions or `case ... of .. -> ...; ... -> ...;` syntax can be used to match a tagged value. 

In [14]:
data Value = Rat (Integer,Integer) | RV Float deriving Show

add (Rat (a,b)) (Rat (c,d)) = Rat (a*d+b*c, b*d)
add (Rat (a,b)) (RV f) = RV (f + (fromInteger a)/fromInteger b)
add (RV f) (Rat (a,b)) = RV (f + (fromInteger a)/fromInteger b)
add (RV g) (RV f) = RV (f + g)

add (Rat(3,4)) (Rat(1,3))
add (add (Rat(3,4)) (Rat(1,3)))  (RV 4)
add (add (Rat(3,4)) (Rat(1,3))) (Rat (21,5))

toReal val = case val of
           RV v -> v
           Rat (a,b) -> fromInteger a/fromInteger b
toReal (Rat (4,13))
toReal (RV 2.2)

Line 4: Redundant bracket
Found:
(fromInteger a) / fromInteger b
Why not:
fromInteger a / fromInteger bLine 5: Redundant bracket
Found:
(fromInteger a) / fromInteger b
Why not:
fromInteger a / fromInteger b

Rat (13,12)

RV 5.0833335

Rat (317,60)

0.30769232

2.2

Function definitions can be written in terms of `case .. of` expressions. Following two definitions are equivalent:

```Haskell
f pa pb = exp1
f pc pd = exp2
f pe pf = exp3

-- vs

f a b = case a,b of
            pa,pb -> exp1
            pc,pd -> exp2
            pe,pf -> exp3

```

## Lists

Haskell lists are defined as recursive data types with a list is either empty (`[]`) or composed by a value and list of following elements (`v : otherelements`). 

```Haskell
empty = []
five = 1 : 2 : 3 : 4 : 5 : []
five' = 1 : (2 : (3 : (4 : (5 : []))))
five'' = [1,2,3,4,5]
```

Haskell `String` values are actually list of `Char` values as:
`"abc" == ['a','b','c']`

all `five, five', five''` above are equivalent. Also mathematical ranges can be defined as:
`[1..5]` which is equivalent to `[1,2,3,4,5]`. 

`[a..b]` = $[a, a+1, a+2,..., b-1, b]$

`[a,b..c]` = $[a, a+d, a+2 d, ..., m]$ where $d=b-a$ and $m$ is the maximum value holding $m = a+n\;d \le c$

In [15]:
[1..30]
[1,6..50]
[30,28..3]

[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]

[1,6,11,16,21,26,31,36,41,46]

[30,28,26,24,22,20,18,16,14,12,10,8,6,4]

List comprehension works like a mathematical set definition as:

`[exp | v <- list1 ]`  it maps each value `v` from `list1` into `exp` 
`[v*v | v <- [1,2,3,4,5]` gives `[1,4,9,16,25]`

it can contain conditions as:

`[v*v | v <- [1,2,3,4,5], mod v 2 == 0]` gives `[4,16]`

values from multiple lists can be combined as cartesian product mappings:

`[a*b | a <- [1,2,3], b <- [5,6]]` will give `[5,6,10,12,15,18]`



In [16]:
[a*a+a | a <- [1..20]]
[a*a | a <- [1..30], a*a > 50]
[(a,b) | a<- [1..4], b <- "ABC"]

[2,6,12,20,30,42,56,72,90,110,132,156,182,210,240,272,306,342,380,420]

[64,81,100,121,144,169,196,225,256,289,324,361,400,441,484,529,576,625,676,729,784,841,900]

[(1,'A'),(1,'B'),(1,'C'),(2,'A'),(2,'B'),(2,'C'),(3,'A'),(3,'B'),(3,'C'),(4,'A'),(4,'B'),(4,'C')]

Lists are recursive values and processing requires recursive functions. Haskell standard library provide many functions:
```Haskell
(!!) :: [a] -> Int -> a                          -- select nth member of 
(++) :: [a] -> [a] -> [a]                        -- concatenate two lists into new one
concat :: Foldable t => t [a] -> [a]             -- concatenate many lists into new one
drop :: Int -> [a] -> [a]                        -- drop first n element of list and return rest
dropWhile :: (a -> Bool) -> [a] -> [a]           -- drop values with a condition and return rest
filter :: (a -> Bool) -> [a] -> [a]              -- filter values and return rest
length :: Foldable t => t a -> Int               -- count number of elements 
map :: (a -> b) -> [a] -> [b]                    -- map apply a function to all and return new list of results
reverse :: [a] -> [a]                            -- reverse the list 
take :: Int -> [a] -> [a]                        -- take first n elements of a list into returned list
takeWhile :: (a -> Bool) -> [a] -> [a]           -- fake first elemtements having condition true
```

Let us implement some of them:

In [17]:
[1,2,3,4] !! 2
[1,2] ++ [3,4]
concat [[1,2,3],[4,5,6],[3,4,5]]
concat ["onur","tolga","sehitoglu"]
drop 4 [1..10]
dropWhile (\x -> x < 7) [1,2,3,9,2,3,1]
filter (\x -> x < 7) [1,2,3,9,2,13,1]
map (\x -> x*x) [1,2,5,6,3,4,6]
reverse "onur tolga sehitoglu"
take 10 [1..1000000000]
takeWhile (\x -> x < 7) [1,2,3,9,2,3,1]


Line 6: Avoid lambda using `infix`
Found:
(\ x -> x < 7)
Why not:
(< 7)Line 7: Avoid lambda using `infix`
Found:
(\ x -> x < 7)
Why not:
(< 7)Line 11: Avoid lambda using `infix`
Found:
(\ x -> x < 7)
Why not:
(< 7)

3

[1,2,3,4]

[1,2,3,4,5,6,3,4,5]

"onurtolgasehitoglu"

...

[1,2,3,2,1]

[1,4,25,36,9,16,36]

"ulgotihes aglot runo"

[1,2,3,4,5,6,7,8,9,10]

[1,2,3]

In [18]:
newlength [] = 0                    -- length of empty list is 0
newlength (_:r) = 1 + newlength r   -- length of rest + 1

plusplus [] second = second         -- concatenate empty list with a second list is the second
plusplus (a:r) second = a : plusplus r second       -- concatenate rest with second and insert value 

newdrop _ [] = []                   -- drop anything from empty list is empty list
newdrop n (v:rest) | n <= 0    = v : rest   -- drop 0 or less element from list is the list itself
                   | otherwise = newdrop (n-1) rest       -- drop one less from the rest

newtake _ [] = []                      -- take anything from empty list is empty list
newtake n (v:rest) | n <= 0    = []    -- no other values required to take
                | otherwise = v : newtake (n-1) rest    -- take one less from rest, insert to the beginning

exex [] _ = error "No more value left"            -- raise an error, trying to get a non-existing value
exex (v:rest) n | n == 0    = v                   -- reached the desired value, return the first
                | otherwise = exex rest (n-1)     -- value is in remainder list, ask with one less
                
newfilter _ [] = []
newfilter func (v:rest) | func v    = v : newfilter func rest
                        | otherwise = newfilter func rest

newlength [4,6,7,3,4,5]
plusplus [1.0, 3.4, 4.2] [1.2,2.3,3.1,4.1]
newdrop 4 "hello world"
newtake 4 "hello world"
exex [7,5,4,5,67,6] 3
newfilter  (> 10) [7,5,4,5,67,6]

Line 4: Use foldr
Found:
plusplus [] second = second
plusplus (a : r) second = a : plusplus r second
Why not:
plusplus r second = foldr (:) second r

6

[1.0,3.4,4.2,1.2,2.3,3.1,4.1]

"o world"

"hell"

5

[67]

More examples. Find the position of an element in a list. Return -1 if not found.

The empty list case is simply evaluates to -1, not found. Otherwise the element is compared against the head of the list and decided on returning new position or continue searching in the rest

In [19]:
index _ [] = -1
index k (v:rest) | k == v    = 0
                 | otherwise = index k rest + 1
                 let inrest = index k rest
                               in if inrest == -1 then -1            -- propagate the error
                                  else 1 + inrest                    -- one plus position in the rest

:t index
index 'c' "Programming Language Concepts"
index 'z' "Programming Language Concepts"

: 

Another option is to carry error information on a special data type. Haskell `Maybe a` can be used for that purpose.
```Haskell
-- either an a valued type or nothing
data Maybe a = Just a | Nothing     -- Nothing is the error case, Just a is the success

```
Data constructors cannot be compared but `case` expression can match them. Case matches the data constructors (see next section) and returns the corresponding expression. Match can contain variables as in function definition.

In [20]:
index' _ [] = Nothing
index' k (v:rest) | k == v  = Just 0
                  | otherwise   =  case index' k rest of
                                    Nothing -> Nothing
                                    Just inrest -> Just (1 + inrest)
:t index'
index' 'c' "Programming Language Concepts"
index' 'z' "Programming Language Concepts"

index' :: forall {t} {a}. (Eq t, Num a) => t -> [t] -> Maybe a

Just 24

Nothing

Reverse definition. You can define reverse simply as:

In [21]:
newreverse [] = []
newreverse (a : rest) = newreverse rest ++ [a]
newreverse [1..30000] !! 29999

1

This implementation has an efficiency problem. Each recursive steps makes a concatenation at the end which require the traversal of the whole list. It adds a value at n, n-1, n-2, n-3, ..., 2, 1 length lists resulting an inefficient calculation. However inserting a value at the beginning is much more cheaper.

Solution is to start from empty list and insert each elements one by one to the head of the list. inserting 1, 2, 3 in the empty list in this order will give 3, 2, 1. So we need an **accumulator list** to calculate the reverse.  Let us write it again.

In [1]:
reverse2' [] second = second                            -- insert empty list into second list is the second
reverse2' (v:rest) second = reverse2' rest (v:second)   -- insert into second and call recursively

reverse2 lst = reverse2' lst []                         -- reverse using an empty accumulator

reverse2 [1..30000] !! 29999

Line 1: Use foldl
Found:
reverse2' [] second = second
reverse2' (v : rest) second = reverse2' rest (v : second)
Why not:
reverse2' rest second = foldl (flip (:)) second rest

1

Substring search. Search for a substring within a string. To write this, let us simplify the problem. If a string starts with the second one, it is a substring of the first. Write this function use it as the minimum case. Then we can check and recursively scan the rest.

**new syntax:** `v@(subpatern)` matches `v` along with pattern enclosed in parenthesis. So you do not need to repeat the `(subpatern)` in the function body.

In [2]:
isprefix [] _ = True     -- empty list is prefix of anything
isprefix _ [] = False    -- if above does not match, first list is not empty, empty list has no non-empty prefix
isprefix (a:ra) (b:rb) | a == b = isprefix ra rb
                       | otherwise = False
                       
substr [] [] = Just 0
substr _ [] = Nothing
substr sub sec@(a:rest) | isprefix sub sec = Just 0
                        | otherwise  = case substr sub rest of
                                        Nothing -> Nothing
                                        Just n -> Just (1 + n)
                                        

In [3]:
substr "dabra" "abracadabra" 
substr "rac" "abracadabra"
substr [4,5] [1..20]
substr "abo" "abracadabra"

Just 6

Just 2

Just 3

Nothing

## Permutation and Powerset

Given list (instead of set), powerset of it should return a list of lists. In order to find all such values, we need to make a recursive definition. If we take `v` out of a list and construct a powerset on remainder list, all members of this list will be natural member of powerset too. In addition, powerset will contain a version with all members also have `v` added:

$P(\{v\} \cup R) \; = \; P(R) \; \cup \; \left\{ \{v\} \cup M \mid M \in P(R) \right\}$


In [13]:
pset [] = [[]]
pset (v:rest) = let nset = pset rest
                    iall _ [] = []
                    iall a (m:r) = (v:m) : iall a r
                in nset ++ iall v nset
                 
:t pset

pset [1,2,3,4]

-- or same function shorter using list comprehension

pset2 [] = [[]]
pset2 (v:rest) = let nset = pset2 rest
                 in nset ++ [ v : m | m <- nset]
                

pset :: forall {t}. [t] -> [[t]]

[[],[4],[3],[3,4],[2],[2,4],[2,3],[2,3,4],[1],[1,4],[1,3],[1,3,4],[1,2],[1,2,4],[1,2,3],[1,2,3,4]]

In [19]:
pset2 "abcdef"

["","f","e","ef","d","df","de","def","c","cf","ce","cef","cd","cdf","cde","cdef","b","bf","be","bef","bd","bdf","bde","bdef","bc","bcf","bce","bcef","bcd","bcdf","bcde","bcdef","a","af","ae","aef","ad","adf","ade","adef","ac","acf","ace","acef","acd","acdf","acde","acdef","ab","abf","abe","abef","abd","abdf","abde","abdef","abc","abcf","abce","abcef","abcd","abcdf","abcde","abcdef"]

Producing all permutation of a list will be similar. We will find the permutation of the smaller. Then the missing value should be inserted all possible locations of each of the smaller permutations.

In [22]:
insertall v [] = [[v]]
insertall v (a:rest) = let restinst = insertall v rest       -- take a out, insert v  (list of lists)
                           insertbeg = v:a:rest              -- v inserted to the beginning (list)
                           witha = [a:m | m <- restinst]     -- insert a back to all of restinst (list of lists)
                       in insertbeg : witha 
:t insertall
insertall 'A' "bcdef"

perm [] = [[]]    -- only one member, empty permutation
perm (v:rest) = let restperm = perm rest          -- permutation of list without v (list of lists)
                --    expandall [] = []             -- need to put v back in all positions
                --    expandall (oneperm:others) = insertall v oneperm ++ expandall others
                -- in expandall restperm
                in concat [ insertall v p | p <- restperm ]
                
:t perm

perm [1,2,3,4]


insertall :: forall {a}. a -> [a] -> [[a]]

["Abcdef","bAcdef","bcAdef","bcdAef","bcdeAf","bcdefA"]

perm :: forall {a}. [a] -> [[a]]

[[1,2,3,4],[2,1,3,4],[2,3,1,4],[2,3,4,1],[1,3,2,4],[3,1,2,4],[3,2,1,4],[3,2,4,1],[1,3,4,2],[3,1,4,2],[3,4,1,2],[3,4,2,1],[1,2,4,3],[2,1,4,3],[2,4,1,3],[2,4,3,1],[1,4,2,3],[4,1,2,3],[4,2,1,3],[4,2,3,1],[1,4,3,2],[4,1,3,2],[4,3,1,2],[4,3,2,1]]

## Trees

Trees can be defined in Haskell with data definitions with disjoint union like:

`data Tree a = ETree | Node a (Tree a) (Tree a)`

or

`data Tree a = Leaf a | Node a (Tree a) (Tree a)`

The second version does not allow empty leaves (i.e. left empty, right has a value). You can add it:

`data Tree a = ETree | Leaf a | Node a (Tree a) (Tree a)`

In this version, `Leaf v` and `Node v ETree ETree` will be semantically equivalent, you need to repeat the pattern in your functions. So third one is not a good choice.

trees of different kinds can be defined as required. For example an **n-ary** tree can be defined with combination of trees and lists as:

`data NTree a = Node a [NTree a]`

Tree algorithms needs to be mathematically defined as the other functions. Let us make a **binary search tree** insertion example.

In [1]:
data Tree a = ETree | Node a (Tree a) (Tree a) deriving Show

insert ETree v = Node v ETree ETree       -- inserting a value to an empty tree
insert (Node a left right) v  | a == v  = (Node v left right)
                              | v < a  = Node a (insert left v) right
                              | otherwise = Node a left (insert right v)

:t insert

insert (insert (insert (insert (insert ETree 3) 5) 2) 6) 4

-- traverse the tree, convert it to a list 
traverse ETree = []
traverse (Node a left right) = traverse left ++ a : traverse right

:t traverse

traverse ( insert (insert (insert (insert (insert ETree 3) 5) 2) 6) 4 )

-- Get all leaves of the tree in a list
leaves ETree = []
leaves (Node a ETree ETree) = [a]
leaves (Node a left right) = leaves left ++ leaves right

:t leaves

leaves ( insert (insert (insert (insert (insert ETree 3) 5) 2) 6) 4 )

insert :: forall {t}. Ord t => Tree t -> t -> Tree t

Node 3 (Node 2 ETree ETree) (Node 5 (Node 4 ETree ETree) (Node 6 ETree ETree))

traverse :: forall {a}. Tree a -> [a]

[2,3,4,5,6]

leaves :: forall {a}. Tree a -> [a]

[2,4,6]

In [7]:
:t ($)
:t (.)

ins v ETree = Node v ETree ETree       -- inserting a value to an empty tree
ins v (Node a left right)  | a == v  = (Node v left right)
                           | v < a  = Node a (ins v left) right
                           | otherwise = Node a left (ins v right)

f x = x*x
g x = x*x*x

f (g 4)
f $ g $ f $ f $ f $ g 3

ins 3 (ins 1 (ins 5 (ins 7 (ins 10 ETree))))

-- f ( g a)  ==  f $ g $ a
(ins 3) $ (ins 1) $ (ins 5) $ (ins 7) $ (ins 10 ETree)
ins 3 $ ins 1 $ ins 5 $ ins 7 $ ins 10 ETree

($) :: forall a b. (a -> b) -> a -> b

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

4096

507528786056415600719754159741696356908742250191663887263627442114881

Node 10 (Node 7 (Node 5 (Node 1 ETree (Node 3 ETree ETree)) ETree) ETree) ETree

Node 10 (Node 7 (Node 5 (Node 1 ETree (Node 3 ETree ETree)) ETree) ETree) ETree

Node 10 (Node 7 (Node 5 (Node 1 ETree (Node 3 ETree ETree)) ETree) ETree) ETree

This example can be used to keep a set of values. Let us slightly change our definitions to store a value for given key. Nodes will contain key,value pairs as tuples instead of a single value. Tree will store `v` value for given `k`. Tree is ordered by `k` values.

In [9]:
-- set :: Tree (a,b) -> a -> b -> Tree (a,b)
-- return the new tree with k,v added to a BST 
set ETree k v = Node (k,v) ETree ETree       -- inserting a (k,v) pair to an empty tree
set (Node (ak,av) left right) k v  | ak == k  = Node (k,v) left right
                                   | k < ak   = Node (ak,av) (set left k v) right
                                   | otherwise = Node (ak,av) left (set right k v)

-- set :: Tree (a,b) -> a -> Maybe b
-- search key in BST and return value if found, else Nothing
get ETree _ = Nothing
get (Node (k,v) left right) sk  | sk == k = Just v
                                | sk < k  = get left sk
                                | otherwise = get right sk
                                
:t set
:t get


x = set (set (set (set (set (set (set ETree "martian" 3) "tweety" 5) "daffy" 2) "elmer" 6) "bugs" 4) 
          "sylvester" 123) "bugs" 66
x

get x "bugs"
get x "onur"

-- path :: Node (a,b) -> a -> [(a,b)]
-- like get but returns all intermediate values in the path

path ETree _ = []
path (Node (k,v) left right) sk  | sk == k = [(k,v)]
                                | sk < k  = (k,v) : path left sk 
                                | otherwise = (k,v) : path right sk



traverse x
leaves x

path x "bugs"
path x "sylvester"
path x "martian"

set :: forall {t1} {t2}. Ord t1 => Tree (t1, t2) -> t1 -> t2 -> Tree (t1, t2)

get :: forall {t} {a}. Ord t => Tree (t, a) -> t -> Maybe a

Node ("martian",3) (Node ("daffy",2) (Node ("bugs",66) ETree ETree) (Node ("elmer",6) ETree ETree)) (Node ("tweety",5) (Node ("sylvester",123) ETree ETree) ETree)

Just 66

Nothing

[("bugs",66),("daffy",2),("elmer",6),("martian",3),("sylvester",123),("tweety",5)]

[("bugs",66),("elmer",6),("sylvester",123)]

[("martian",3),("daffy",2),("bugs",66)]

[("martian",3),("tweety",5),("sylvester",123)]

[("martian",3)]

Make a range query. Given two values, find all values matching their keys in $[a,b]$ closed interval, $a \le b$. Gets a tree and two key values, returns a list of values. This will be an example of merging results from recursion. Queries repeated for left and right will be merged into the result. There are following cases, assuming current node key is $k$:

* $a > k$ : In this case, there is no value in left or current node, result of right query is returned
* $b < k$ : Similarly no value from right or current node, left query is returned.
* $a = k$ or $b = k$ are special cases of above two.
* $a < k < b$ : in this case query is sent to left as $[a,k]$ and to right as $[k,b]$ and results merged with current node values.

In [10]:
inrange ETree _ _ = []           -- empty tree, empty result
inrange (Node (k,v) left right) a b  | a > k = inrange right a b
                                     | a == k = v : inrange right a b
                                     | b < k = inrange left a b
                                     | b == k =  inrange left a b ++ [v]    -- append end to keep the ordering
                                     | otherwise = let lq = inrange left a k
                                                       rq = inrange right k b
                                                    in lq ++ v : rq
                                                    
:t inrange

inrange x "c" "h"
inrange x "martian" "tweety"
inrange x "a" "z"
inrange x "" "martian"
inrange x "z" "z"
inrange x "elmer" "elmer"

inrange :: forall {a1} {a2}. Ord a1 => Tree (a1, a2) -> a1 -> a1 -> [a2]

[2,6]

[3,123,5]

[66,2,6,3,123,5]

[66,2,6,3]

[]

[6]

Let us try a more complex traversal of the tree. Find all possible paths from root to all leaves. Gets a BST and returns a list of lists. Each member will be a path. For example `["elmer","tweety","martian"]` is a path from root to `"martian"` node.

In this example we need to find all paths for left and right children, then insert current node to the beginning.

In [21]:
allpaths ETree = []               -- no path. try also returning [[]] here
allpaths (Node (k,_) ETree ETree) = [[k]]                   -- leaf node
allpaths (Node (k,_) left right) = 
        let apl = allpaths left
            apr = allpaths right
           -- insertall _ [] = []   -- insert v to beginning  of all list members of the list
           -- insertall v (lh:lrest) = (v:lh) : insertall v lrest
        -- in insertall k apl ++ insertall k apr
        in [k:lst | lst <- apl ++ apr ] -- ++ [ k:lst | lst <- apr]

:t allpaths

allpaths x

-- what is this? We will talk about higher order functions later
map (path x . fst) (leaves x)
-- Higher order functions. Functions that gets functions as arguments "map"                      

allpaths :: forall {a} {b}. Tree (a, b) -> [[a]]

[["martian","daffy","bugs"],["martian","daffy","elmer"],["martian","tweety","sylvester"]]

[[("martian",3),("daffy",2),("bugs",66)],[("martian",3),("daffy",2),("elmer",6)],[("martian",3),("tweety",5),("sylvester",123)]]

[("martian",3),("daffy",2),("bugs",66)]

In [25]:
:t (.)

f x = x*x
g x = x+5

(f . g) 4
(g . f ) 4

(.) :: forall b c a. (b -> c) -> (a -> b) -> a -> c

81

21

In [37]:
path  x (fst ("sylvester",66))

Line 1: Evaluate
Found:
fst ("sylvester", 66)
Why not:
"sylvester"

[("martian",3),("tweety",5),("sylvester",123)]

In [34]:
leaves x

[("bugs",66),("elmer",6),("sylvester",123)]

In [41]:
foldr (+) 0 [1,4,5,7,8,3]
reduce func s [] = s
reduce func s (el:rest) = func el (reduce func s rest)
reduce (*) 1 [1..7]

Line 1: Use sum
Found:
foldr (+) 0
Why not:
sumLine 2: Use foldr
Found:
reduce func s [] = s
reduce func s (el : rest) = func el (reduce func s rest)
Why not:
reduce func s rest = foldr func s rest

28

5040

In [45]:
foldl (-) 0 [1,4,5,7,8,3]
foldr (-) 0 [1,4,5,71,8,3]


-28

-64

In [46]:
iter f s 0 = s
iter f s n = f (iter f s (n-1))

In [52]:
power2 = iter (* 2) 1 

power2 8

256

In [57]:
:t filter

filter :: forall a. (a -> Bool) -> [a] -> [a]

In [62]:
mult x = iter (+ x) 0
:t mult
mult 3 5

mult :: forall {t1} {t2}. (Eq t1, Num t1, Num t2) => t2 -> t1 -> t2

15

In [63]:
ifib n = let f (x,y) = (y, x+y)
             (a,b) = iter f (0,1) n
         in b

In [69]:
ifib 100

573147844013817084101

In [73]:
qsort _ [] = []
qsort comp (x:rest) = qsort comp (filter (comp x) rest) ++ x : qsort comp (filter (\y->(not (comp x y))) rest)

Line 2: Avoid lambda
Found:
\ y -> (not (comp x y))
Why not:
not . comp xLine 2: Redundant bracket
Found:
\ y -> (not (comp x y))
Why not:
\ y -> not (comp x y)

In [74]:
qsort (<) [1,7,5,6,8,90,5,4,5,7,8,9]

[90,9,8,8,7,7,6,5,5,5,4,1]

In [75]:
qsort (>) [1,7,5,6,8,90,5,4,5,7,8,9]

[1,4,5,5,5,6,7,7,8,8,9,90]